### 🔍 Objective:
This project will **introduce you to CNNs**.

### 📌 What You’ll Do:
1. Define suitable transforms/augmentations for your `train` and `test` images.
2. Pass these images into PyTorch `DataLoaders` for batch processing.
3. Implement `CNN` class architecture for pneumonia image classification.
4. Train and validate your model.

💡 **PLEASE PLEASE PLEASE look things up!!! This is YOUR learning experience.**

---

In [1]:
import torch
import kagglehub
import torch.nn as nn

path = kagglehub.dataset_download("paultimothymooney/chest-xray-pneumonia")

train_path = f'{path}/chest_xray/train/'
test_path = f'{path}/chest_xray/test/'

#### 📌 ***TASK 1 - DATA PREPROCESSING***

Define image augmentations in the cell below using two variables:  

- **`transform_train`**: Stores transforms for training images. You can include any augmentations you prefer.  
- **`transform_test`**: Stores transforms for your test images. As a best practice, limit these transformations to only the essential ones from `transform_train`.

Lastly, be sure to convert all images to [tensors](https://www.perplexity.ai/search/i-m-a-student-at-naiss-mlb-and-_EL_nBO9TS694cbTEl5M.A) via the `transforms.ToTensor()` transform. Don't know transforms? [Click here](https://pytorch.org/vision/stable/transforms.html).

In [2]:
from torchvision import transforms

transform_train = transforms.Compose([
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(10),
    transforms.RandomResizedCrop(224, scale=(0.8, 1.0)),
    transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.1),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])
])

transform_test = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])
])

Now, we'll pass our images and transforms into a `DataLoader`, which allows us to train our model in batches.
Most of the code is done for you, but [click here](https://www.perplexity.ai/search/i-m-a-student-at-naiss-mlb-and-_EL_nBO9TS694cbTEl5M.A) to learn more.

In [3]:
from torchvision.datasets import ImageFolder
from torch.utils.data import DataLoader

num_batches = 32  

train_dataset = ImageFolder(root=train_path, transform=transform_train) 
train_loader = DataLoader(dataset=train_dataset, batch_size=num_batches, shuffle=True)

test_dataset = ImageFolder(root=test_path, transform=transform_test) 
test_loader = DataLoader(dataset=test_dataset, batch_size=num_batches, shuffle=True)

#### 📌 ***TASK 2 - CNN Architecture***

This is where you have all the creative freedom in the world. Here are some good questions to ask yourself:

- How many [channels](https://www.perplexity.ai/search/i-m-a-student-at-naiss-mlb-wha-49AG77e4Qp2e7EkARdFsTA) should go into the input layer?
- What measures can I take to avoid [overfitting](https://www.perplexity.ai/search/i-m-a-student-at-naiss-mlb-wha-YdAbhqQzRZaEq39BEQzA6w)?
- What matters to me? (Training Speed / Performance tradeoffs)
- **CONVOLUTION. ACTIVATION FUNCTION. POOLING!!!** 📢📢📢

Not comfortable with PyTorch? [Here](https://youtu.be/mozBidd58VQ?si=TE2_81TEQko1eDXT). Go and make me the best [CNN](https://www.datacamp.com/tutorial/introduction-to-convolutional-neural-networks-cnns) I've ever seen :)

In [4]:
import torch
import torch.nn

# Remember what Pooling does to feature maps!!!
image_width = 224  
image_height = 224  

class CNNModel(nn.Module):
    def __init__(self, num_classes):
        super(CNNModel, self).__init__()

        self.conv1 = nn.Conv2d(in_channels=3, out_channels=32, kernel_size=3, padding=1)
        self.conv2 = nn.Conv2d(in_channels=32, out_channels=64, kernel_size=3, padding=1)
        self.conv3 = nn.Conv2d(in_channels=64, out_channels=128, kernel_size=3, padding=1)

        self.pool = nn.MaxPool2d(kernel_size=2, stride=2)

        self.fc1 = nn.Linear(128 * (image_width // 8) * (image_height // 8), 512)
        self.fc2 = nn.Linear(512, num_classes)

        self.dropout = nn.Dropout(0.5)

    def forward(self, x):
        x = self.pool(torch.relu(self.conv1(x)))
        x = self.pool(torch.relu(self.conv2(x)))
        x = self.pool(torch.relu(self.conv3(x)))

        x = torch.flatten(x, start_dim=1)
        x = torch.relu(self.fc1(x))
        x = self.dropout(x)
        x = self.fc2(x)

        return x

num_classes = 10  
model = CNNModel(num_classes)

#### 📌 ***TASK 3 - DEFINE TRAIN FUNCTION***  

Define `process_forward_phase` and `train` to update model weights with each new [epoch/iteration](https://www.perplexity.ai/search/i-m-a-curious-naiss-mlb-studen-7SJECNYrS1iYxUR032dp7A). Here are the steps:

- **Forward pass:** Here, our batch is taken through the network to output a prediction (Normal/Pneumonia)
- **Backward pass:** The model goes "What's our loss? Hmmm... Not quite what I want. This means my `weights` aren't adjusted properly. Let me propagate my `loss` backward in hopes of correcting my weights."

We use **`f1_score`** as the primary metric and also display **`accuracy`** for comparison. Most steps are outlined for you—just follow the structure provided!

In [ ]:
from tqdm import tqdm # Visualize training progress
from sklearn.metrics import f1_score, accuracy_score

def process_forward_pass(model, batch, criterion):
    """
    This is a helper function to abstract the "forward"
    phase of the training loop. This function also returns
    the loss, predictions, and labels seen in the batch.
    """

    images, labels = batch
    labels = labels.long()

    outputs = model(images)
    loss = criterion(outputs, labels()) 

    preds = torch.argmax(outputs, dim=1)

    return loss, preds, labels

def train(model, train_loader, criterion, optimizer, epochs):
    model.train() # Set model to training mode

    for epoch in range(epochs): # Specify the number of iterations to train for
        all_preds, all_labels = [], []

        for batch in tqdm(train_loader, desc=f"Epoch {epoch + 1}/{epochs}"):
            optimizer.zero_grad()
            loss, preds, labels = process_forward_pass(model, batch, criterion)

            loss.backward()
            optimizer.step() # (backward phase, upd. weights)

            all_preds.extend(preds.numpy())
            all_labels.extend(labels.numpy())

        accuracy = accuracy_score(all_labels, all_preds)
        f1 = f1_score(all_labels, all_preds, average='weighted')

        print(f"Acc={accuracy:.2f}%, F1={f1:.4f}")

After your model trains, you want to see how well it performs on **unseen data.** Meaning, if this were a live hospital NEEDING your predictions to classify patients with pneumonia, how well would it do?


You simply have to run this cell; all the code is implemented for you (Assuming `process_forward_phase` works fine). 😊

In [7]:
def test(model, test_loader, criterion):
    model.eval()
    all_preds, all_labels = [], []

    with torch.no_grad():
        for batch in test_loader:
            loss, preds, labels = process_forward_pass(model, batch, criterion)
            all_preds.extend(preds.numpy())
            all_labels.extend(labels.numpy())

    accuracy = accuracy_score(all_labels, all_preds)
    f1 = f1_score(all_labels, all_preds, average="weighted")

    print(f"Final Test Results: Acc={accuracy:.2f}%, F1={f1:.4f}")

#### 📌***TASK 4 - TRAIN MODEL***

We're close!!! We simply need to instantiate the `model`, define a suitable `criterion` (loss), and use an `optimizer` (thing to speed up backpropagation).

In [10]:
import torch.optim as optim

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001, weight_decay=1e-5) # Weight decay is optional. But is it? 🤔

train(model, train_loader, criterion, optimizer, epochs=10)

Epoch 1/10: 100%|██████████| 163/163 [05:56<00:00,  2.19s/it]


Acc=0.94%, F1=0.9595


Epoch 2/10: 100%|██████████| 163/163 [05:40<00:00,  2.09s/it]


Acc=0.94%, F1=0.9628


Epoch 3/10: 100%|██████████| 163/163 [05:50<00:00,  2.15s/it]


...

Acc=0.96%, F1=0.9709


Epoch 9/10: 100%|██████████| 163/163 [05:51<00:00,  2.16s/it]


Acc=0.96%, F1=0.9725


Epoch 10/10: 100%|██████████| 163/163 [05:58<00:00,  2.20s/it]

Acc=0.96%, F1=0.9719


Last step: evaluate your model's performance. Remember, you get **1,000,000** brownie points 🍫 if you beat Adam's **`f1_score:`0.8549**.

In [11]:
test(model, test_loader, criterion)

Final Test Results: Acc=0.68%, F1=0.5933
